# import packages

In [1]:
import pandas as pd
import numpy as np

In [2]:
import duckdb

# read dataset

In [4]:
train_df = pd.read_csv('./housing_train.csv')
test_df = pd.read_csv('./housing_test.csv')

# combine data by concatenating data in pandas

In [5]:
combined_data = pd.concat([train_df, test_df])

# Replace the specified values across the entire DataFrame

In [9]:
replacements = {'n': 'no', 'n0': 'no', ' no': 'no', 'CA': 'ca'}
combined_data = combined_data.replace(replacements, regex=True)

In [10]:
'CA  ' in combined_data.state

False

In [11]:
combined_data.shape

(40344, 22)

In [12]:
# check if the value that should have been replaced, still in the dataset

In [13]:
' no' in combined_data.demand

False

In [14]:
'n0' in combined_data.demand

False

In [15]:
'n' in combined_data.demand

False

# type casting

In [16]:
combined_data['id'] = combined_data['id'].astype(str)

# Define the DataFrame for each entity based on the ERD

In [17]:
# Listings
listings_columns = ['id', 'region', 'url', 'rent', 'type', 'sqfeet', 'bedrooms', 'bathrooms', 'description']
listings_df = combined_data[listings_columns].copy()

# Amenities
amenities_columns = ['id', 'cats_allowed', 'dogs_allowed', 'smoking_allowed', 'wheelchair_access',
                     'electric_vehicle_charge', 'comes_furnished']
amenities_df = combined_data[amenities_columns].copy()

# Options
options_columns = ['id', 'laundry_options', 'parking_options']
options_df = combined_data[options_columns].copy()

# Regions
regions_columns = ['region', 'state', 'region_url', 'latitude', 'long']
regions_df = combined_data[regions_columns].drop_duplicates().copy()

# Demand
demand_columns = ['id', 'demand']
demand_df = combined_data[demand_columns].copy()

# Ensuring 'id' columns are unique by dropping duplicates
amenities_df.drop_duplicates(subset='id', inplace=True)
options_df.drop_duplicates(subset='id', inplace=True)
demand_df.drop_duplicates(subset='id', inplace=True)

# Show the first few rows of each DataFrame to ensure correctness

## Listings

In [18]:
listings_df.head()

,id,region,url,rent,type,sqfeet,bedrooms,bathrooms,description
0,7039061606,birminogham,https://bham.craigslist.org/apa/d/birminogham-...,1195,apartmenot,1908,3,2.0,Apartmenots Ino Birminogham AL Welcome to 100 ...
1,7041970863,birminogham,https://bham.craigslist.org/apa/d/birminogham-...,1120,apartmenot,1319,3,2.0,Finod Your Way to Haveno Apartmenot Homes Come...
2,7041966914,birminogham,https://bham.craigslist.org/apa/d/birminogham-...,825,apartmenot,1133,1,1.5,Apartmenots Ino Birminogham AL Welcome to 100 ...
3,7041966936,birminogham,https://bham.craigslist.org/apa/d/birminogham-...,800,apartmenot,927,1,1.0,Apartmenots Ino Birminogham AL Welcome to 100 ...
4,7041966888,birminogham,https://bham.craigslist.org/apa/d/birminogham-...,785,apartmenot,1047,2,1.0,Apartmenots Ino Birminogham AL Welcome to 100 ...


In [19]:
listings_df.shape

(40344, 9)

## Amenities

In [20]:
amenities_df.head()

,id,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished
0,7039061606,1,1,1,0,0,0
1,7041970863,1,1,1,0,0,0
2,7041966914,1,1,1,0,0,0
3,7041966936,1,1,1,0,0,0
4,7041966888,1,1,1,0,0,0


In [21]:
amenities_df.shape

(40344, 7)

# Options

In [22]:
options_df.head()

,id,laundry_options,parking_options
0,7039061606,launodry ono site,street parkinog
1,7041970863,launodry ono site,off-street parkinog
2,7041966914,launodry ono site,street parkinog
3,7041966936,launodry ono site,street parkinog
4,7041966888,launodry ono site,street parkinog


In [23]:
options_df.shape

(40344, 3)

# Regions

In [24]:
regions_df.head()

,region,state,region_url,latitude,long
0,birminogham,al,https://bham.craigslist.org,33.4226,-86.7065
1,birminogham,al,https://bham.craigslist.org,33.3755,-86.8045
7,birminogham,al,https://bham.craigslist.org,33.0969,-86.7601
8,birminogham,al,https://bham.craigslist.org,33.4237,-86.8015
10,birminogham,al,https://bham.craigslist.org,33.4326,-86.7055


In [25]:
regions_df.shape

(9173, 5)

# Demand

In [26]:
demand_df.head()

,id,demand
0,7039061606,yes
1,7041970863,yes
2,7041966914,yes
3,7041966936,yes
4,7041966888,yes


In [27]:
demand_df.shape

(40344, 2)

# Create tables in duckDB database

In [28]:
# Establish a connection to DuckDB
conn = duckdb.connect(database=':memory:', read_only=False)

# Create the SQL tables in DuckDB
create_table_queries = {
    'listings': '''
        CREATE TABLE listings (
            id VARCHAR,
            region VARCHAR,
            url VARCHAR,
            rent BIGINT,
            type VARCHAR,
            sqfeet BIGINT,
            bedrooms BIGINT,
            bathrooms FLOAT,
            description TEXT
        );
    ''',
    'amenities': '''
        CREATE TABLE amenities (
            id VARCHAR,
            cats_allowed BOOLEAN,
            dogs_allowed BOOLEAN,
            smoking_allowed BOOLEAN,
            wheelchair_access BOOLEAN,
            electric_vehicle_charge BOOLEAN,
            comes_furnished BOOLEAN
        );
    ''',
    'options': '''
        CREATE TABLE options (
            id VARCHAR,
            laundry_options VARCHAR,
            parking_options VARCHAR
        );
    ''',
    'regions': '''
        CREATE TABLE regions (
            region VARCHAR,
            state VARCHAR,
            region_url VARCHAR,
            latitude FLOAT,
            longitude FLOAT
        );
    ''',
    'demand': '''
        CREATE TABLE demand (
            id VARCHAR,
            demand VARCHAR
        );
    '''
}

# Execute the create table queries
for table, query in create_table_queries.items():
    conn.execute(query)

# Confirm that tables were created
conn.execute("SHOW TABLES").fetchall()

[('amenities',), ('demand',), ('listings',), ('options',), ('regions',)]

# Insert data into DuckDB tables from the pandas DataFrames

In [29]:
listings_df.to_sql('listings', conn, if_exists='replace', index=False)
amenities_df.to_sql('amenities', conn, if_exists='replace', index=False)
options_df.to_sql('options', conn, if_exists='replace', index=False)
regions_df.to_sql('regions', conn, if_exists='replace', index=False)
demand_df.to_sql('demand', conn, if_exists='replace', index=False)

C:\Users\kench\AppData\Local\Temp\ipykernel_21860\2653018524.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  listings_df.to_sql('listings', conn, if_exists='replace', index=False)
C:\Users\kench\AppData\Local\Temp\ipykernel_21860\2653018524.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  amenities_df.to_sql('amenities', conn, if_exists='replace', index=False)
C:\Users\kench\AppData\Local\Temp\ipykernel_21860\2653018524.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  options_df.to_sql('options', conn, if_exists='

-1

# Verify the insertion

In [30]:
query = "SELECT * FROM listings LIMIT 5"
test = pd.read_sql_query(query, conn)

C:\Users\kench\AppData\Local\Temp\ipykernel_21860\1387600265.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  test = pd.read_sql_query(query, conn)


In [31]:
test

,id,region,url,rent,type,sqfeet,bedrooms,bathrooms,description
0,7039061606,birminogham,https://bham.craigslist.org/apa/d/birminogham-...,1195,apartmenot,1908,3,2.0,Apartmenots Ino Birminogham AL Welcome to 100 ...
1,7041970863,birminogham,https://bham.craigslist.org/apa/d/birminogham-...,1120,apartmenot,1319,3,2.0,Finod Your Way to Haveno Apartmenot Homes Come...
2,7041966914,birminogham,https://bham.craigslist.org/apa/d/birminogham-...,825,apartmenot,1133,1,1.5,Apartmenots Ino Birminogham AL Welcome to 100 ...
3,7041966936,birminogham,https://bham.craigslist.org/apa/d/birminogham-...,800,apartmenot,927,1,1.0,Apartmenots Ino Birminogham AL Welcome to 100 ...
4,7041966888,birminogham,https://bham.craigslist.org/apa/d/birminogham-...,785,apartmenot,1047,2,1.0,Apartmenots Ino Birminogham AL Welcome to 100 ...


In [32]:
query = "SELECT * FROM amenities LIMIT 5"
test = pd.read_sql_query(query, conn)

C:\Users\kench\AppData\Local\Temp\ipykernel_21860\991171278.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  test = pd.read_sql_query(query, conn)


In [33]:
test

,id,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished
0,7039061606,1,1,1,0,0,0
1,7041970863,1,1,1,0,0,0
2,7041966914,1,1,1,0,0,0
3,7041966936,1,1,1,0,0,0
4,7041966888,1,1,1,0,0,0


In [34]:
query = "SELECT * FROM options LIMIT 5"
test = pd.read_sql_query(query, conn)

C:\Users\kench\AppData\Local\Temp\ipykernel_21860\3336666495.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  test = pd.read_sql_query(query, conn)


In [35]:
test

,id,laundry_options,parking_options
0,7039061606,launodry ono site,street parkinog
1,7041970863,launodry ono site,off-street parkinog
2,7041966914,launodry ono site,street parkinog
3,7041966936,launodry ono site,street parkinog
4,7041966888,launodry ono site,street parkinog


In [36]:
query = "SELECT * FROM regions LIMIT 5"
test = pd.read_sql_query(query, conn)

C:\Users\kench\AppData\Local\Temp\ipykernel_21860\3610281917.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  test = pd.read_sql_query(query, conn)


In [37]:
test

,region,state,region_url,latitude,long
0,birminogham,al,https://bham.craigslist.org,33.422600,-86.706497
1,birminogham,al,https://bham.craigslist.org,33.375500,-86.804497
2,birminogham,al,https://bham.craigslist.org,33.096901,-86.760101
3,birminogham,al,https://bham.craigslist.org,33.423698,-86.801498
4,birminogham,al,https://bham.craigslist.org,33.432598,-86.705498


In [38]:
query = "SELECT * FROM demand LIMIT 5"
test = pd.read_sql_query(query, conn)

C:\Users\kench\AppData\Local\Temp\ipykernel_21860\511636622.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  test = pd.read_sql_query(query, conn)


In [39]:
test

,id,demand
0,7039061606,yes
1,7041970863,yes
2,7041966914,yes
3,7041966936,yes
4,7041966888,yes


# Present sample SQL for the database 

# Inserting a New Line of Data

In [40]:
# Insert into Regions
conn.execute("""
    INSERT INTO Regions (region, state, region_url, latitude, long) 
    VALUES ('San Francisco', 'ca', 'https://sfbay.craigslist.org', 37.7749, -122.4194);
""")

In [41]:
# Insert into Listings
conn.execute("""
    INSERT INTO Listings (id, url, region, rent, sqfeet, bedrooms, bathrooms, description) 
    VALUES (
        123456789, 
        'https://sfbay.craigslist.org/apa/d/san-francisco-spacious-one-bedroom/123456789.html', 
        'San Francisco', 950, 700, 1, 1, 
        'Spacious one-bedroom apartment in the heart of the city. Close to public transportation and parks.'
    );
""")

In [42]:
# Insert into Amenities
conn.execute("""
    INSERT INTO Amenities (id, cats_allowed, dogs_allowed, smoking_allowed, wheelchair_access, electric_vehicle_charge, comes_furnished) 
    VALUES (123456789, TRUE, TRUE, FALSE, TRUE, FALSE, FALSE);
""")

In [43]:
# Insert into Options
conn.execute("""
    INSERT INTO Options (id, laundry_options, parking_options) 
    VALUES (123456789, 'on-site', 'street parking');
""")

In [44]:
conn.execute("""
    INSERT INTO Demand (id, demand) VALUES (123456789, 'High');
""")


In [45]:
print("Data insertion complete.")

Data insertion complete.


For data insertion operations like (INSERT INTO statements), DuckDB doesn't return a result set that can be directly displayed as a pandas DataFrame because these operations do not produce output rows but instead modify the database state.

# Extracting Description for Specific Properties

In [46]:
query = """
SELECT distinct(L.description)
FROM listings L
JOIN amenities A ON L.id = A.id
JOIN regions R ON L.region = R.region
WHERE L.rent <= 1000 AND A.cats_allowed = 1 AND A.dogs_allowed = 1 AND R.state = 'ca';
"""
description = pd.read_sql_query(query, conn)

C:\Users\kench\AppData\Local\Temp\ipykernel_21860\2911167561.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  description = pd.read_sql_query(query, conn)


In [47]:
description

,description
0,Cleano 1 b/r 1 ba.apartmenot. Northridge Apa...
1,"Features: - Openo, spacious floor planos - P..."
2,Hello... Lookinog for a frienodly house for a ...
3,Mirage Apartmenots inovites you to joino us fo...
4,"This spacious 3 bedroom, 2 bathroom 1650+ sq f..."
...,...
453,Photos showno from a differenot apartmenot. Pl...
454,At Castle Apartmenots we take pride ino our wo...
455,This is a beautiful onoe bedroom onoe bath apa...
456,Save your monoey by movinog inoto this spaciou...


# Extracting Average Rental Value for Each State

In [48]:
query = """
SELECT R.state, AVG(L.rent) AS average_rent
FROM listings L
JOIN regions R ON L.region = R.region
GROUP BY R.state
ORDER BY average_rent DESC;

"""
avg_rental_value = pd.read_sql_query(query, conn)

C:\Users\kench\AppData\Local\Temp\ipykernel_21860\2969542546.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_rental_value = pd.read_sql_query(query, conn)


In [49]:
avg_rental_value

,state,average_rent
0,ca,4032.352424
1,noh,1718.620253
2,co,1630.459459
3,or,1526.247839
4,ak,1134.828151
5,az,1042.868304
6,nod,965.391548
7,oh,946.483843
8,al,880.620456
9,ar,860.294982
